In [1]:
import findspark
findspark.init()

import glob as g
import datetime
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import *

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.appName("YouTube Data Analysis").getOrCreate()
print(spark.sparkContext.appName)

pyspark-shell


In [2]:
csv_df=spark.read.csv(header=True,inferSchema=True,path=['CAvideos.csv','GBvideos.csv','USvideos.csv'])

In [3]:
csv_df.printSchema()
csv_df.show(5)

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)

+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|

In [4]:
json_df=spark.read.option("multiline","true").json(['CA_category_id.json','GB_category_id.json','US_category_id.json'])

In [5]:
json_df.printSchema()
json_df.show()
#json_df.show(truncate=False)

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- assignable: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |-- kind: string (nullable = true)

+--------------------+--------------------+--------------------+
|                etag|               items|                kind|
+--------------------+--------------------+--------------------+
|"m2yskBQFythfE4ir...|[{"m2yskBQFythfE4...|youtube#videoCate...|
|"m2yskBQFythfE4ir...|[{"m2yskBQFythfE4...|youtube#videoCate...|
|"ld9biNPKjAjgjV7E...|[{"ld9biNPKjAjgjV...|youtube#videoCate...|
+--------------------+--------------------+--------------------+



In [6]:
display(json_df)

DataFrame[etag: string, items: array<struct<etag:string,id:string,kind:string,snippet:struct<assignable:boolean,channelId:string,title:string>>>, kind: string]

In [7]:
print("Shape of the records in csv file",csv_df.count(),len(csv_df.columns))

Shape of the records in csv file 136992 16


In [8]:
#Remove irrelevant columns.
csv_df=csv_df.drop('thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','description')

In [9]:
csv_df.describe("video_id","title","likes").show()

+-------+----------------------------+--------------------+--------------------+
|summary|                    video_id|               title|               likes|
+-------+----------------------------+--------------------+--------------------+
|  count|                      136992|              123151|              121021|
|   mean|                        null|   393.7241379310345|   81943.00448876154|
| stddev|                        null|  123.66368463114675|   254409.1223905117|
|    min|                      #DRAMA|                  36|       Abdullah Ejaz|
|    max|◎追蹤【三立新聞網Instagra...|🚨Active Shooter ...|Shajeer Uddin as ...|
+-------+----------------------------+--------------------+--------------------+



In [10]:
csv_df.dtypes

[('video_id', 'string'),
 ('trending_date', 'string'),
 ('title', 'string'),
 ('channel_title', 'string'),
 ('category_id', 'string'),
 ('publish_time', 'string'),
 ('tags', 'string'),
 ('views', 'string'),
 ('likes', 'string'),
 ('dislikes', 'string'),
 ('comment_count', 'string')]

In [11]:
csv_df.summary().show(5)

+-------+--------+------------------+------------------+--------------------+--------------------+------------+-------------------+-------------------+-----------------+-----------------+--------------------+
|summary|video_id|     trending_date|             title|       channel_title|         category_id|publish_time|               tags|              views|            likes|         dislikes|       comment_count|
+-------+--------+------------------+------------------+--------------------+--------------------+------------+-------------------+-------------------+-----------------+-----------------+--------------------+
|  count|  136992|            123572|            123151|              122901|              122352|      122087|             121149|             121047|           121021|           120978|              120975|
|   mean|    null|1901.4117647058824| 393.7241379310345|                null|  19.238018335859277|        null|               null|  3094371.187409935|81943.0044887

In [12]:
csv_df.toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"""Eminem""|""Walk""|""On""|""Water""|""Aftermath/Shady/...",17158579,787425,43420,125882
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"""plush""|""bad unboxing""|""unboxing""|""fan mail""|""...",1014651,127794,1688,13030
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146035,5339,8181
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",2095828,132239,1989,17518
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"""edsheeran""|""ed sheeran""|""acoustic""|""live""|""co...",33523622,1634130,21082,85067
...,...,...,...,...,...,...,...,...,...,...,...
136987,l884wKofd54,18.14.06,Enrique Iglesias - MOVE TO MIAMI (Official Vid...,EnriqueIglesiasVEVO,10,2018-05-09T07:00:01.000Z,"""Enrique Iglesias feat. Pitbull""|""MOVE TO MIAM...",25066952,268088,12783,9933
136988,IP8k2xkhOdI,18.14.06,Jacob Sartorius - Up With It (Official Music V...,Jacob Sartorius,10,2018-05-11T17:09:16.000Z,"""jacob sartorius""|""jacob""|""up with it""|""jacob ...",1492219,61998,13781,24330
136989,Il-an3K9pjg,18.14.06,Anne-Marie - 2002 [Official Video],Anne-Marie,10,2018-05-08T11:05:08.000Z,"""anne""|""marie""|""anne-marie""|""2002""|""two thousa...",29641412,394830,8892,19988
136990,-DRsfNObKIQ,18.14.06,Eleni Foureira - Fuego - Cyprus - LIVE - First...,Eurovision Song Contest,24,2018-05-08T20:32:32.000Z,"""Eurovision Song Contest""|""2018""|""Lisbon""|""Cyp...",14317515,151870,45875,26766


In [13]:
from pyspark.sql.types import StructType, StructField
schema = StructType([StructField("video_id", StringType(), True)\
                       ,StructField("trending_date", DateType(), True)\
                       ,StructField("title", StringType(), True)\
                       ,StructField("channel_title", StringType(), True)\
                       ,StructField("category_id", IntegerType(), True)\
                       ,StructField("publish_time", TimestampType(), True)\
                       ,StructField("tags", StringType(), True)\
                       ,StructField("views", IntegerType(), True)\
                       ,StructField("likes", IntegerType(), True)\
                       ,StructField("dislikes", IntegerType(), True)\
                       ,StructField("comment_count", IntegerType(), True)\
                       ,StructField("thumbnail_link", StringType(), True)\
                       ,StructField("comments_disabled", StringType(), True)\
                       ,StructField("ratings_disabled", StringType(), True)\
                       ,StructField("video_error_or_removed", StringType(), True)\
                       ,StructField("description", StringType(), True)])
schema

StructType(List(StructField(video_id,StringType,true),StructField(trending_date,DateType,true),StructField(title,StringType,true),StructField(channel_title,StringType,true),StructField(category_id,IntegerType,true),StructField(publish_time,TimestampType,true),StructField(tags,StringType,true),StructField(views,IntegerType,true),StructField(likes,IntegerType,true),StructField(dislikes,IntegerType,true),StructField(comment_count,IntegerType,true),StructField(thumbnail_link,StringType,true),StructField(comments_disabled,StringType,true),StructField(ratings_disabled,StringType,true),StructField(video_error_or_removed,StringType,true),StructField(description,StringType,true)))

In [14]:
csv_df=csv_df.withColumn("publish_date",to_date(csv_df.publish_time))
csv_df.select('publish_date')
csv_df.show(5)

+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+--------+-------+--------+-------------+------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|                tags|   views|  likes|dislikes|comment_count|publish_date|
+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+--------+-------+--------+-------------+------------+
|n1WpP7iowLc|     17.14.11|Eminem - Walk On ...|   EminemVEVO|         10|2017-11-10T17:00:...|"Eminem"|"Walk"|"...|17158579| 787425|   43420|       125882|  2017-11-10|
|0dBIkQ4Mz1M|     17.14.11|PLUSH - Bad Unbox...|    iDubbbzTV|         23|2017-11-13T17:00:...|"plush"|"bad unbo...| 1014651| 127794|    1688|        13030|  2017-11-13|
|5qpjK5DgCt4|     17.14.11|Racist Superman |...| Rudy Mancuso|         23|2017-11-12T19:05:...|"racist superman"...| 3191434| 146035|    5339|        

In [16]:
csv_df.dtypes

[('video_id', 'string'),
 ('trending_date', 'string'),
 ('title', 'string'),
 ('channel_title', 'string'),
 ('category_id', 'string'),
 ('publish_time', 'string'),
 ('tags', 'string'),
 ('views', 'string'),
 ('likes', 'string'),
 ('dislikes', 'string'),
 ('comment_count', 'string'),
 ('publish_date', 'date')]

In [17]:
import pyspark.sql.functions as f
csv_df=csv_df.withColumn('publish_time',to_timestamp(csv_df.publish_time))
csv_df.select(f.date_format("publish_time","HH:mm:ss").alias("publish_time"))
csv_df.show(5)
csv_df.toPandas()

+-----------+-------------+--------------------+-------------+-----------+-------------------+--------------------+--------+-------+--------+-------------+------------+
|   video_id|trending_date|               title|channel_title|category_id|       publish_time|                tags|   views|  likes|dislikes|comment_count|publish_date|
+-----------+-------------+--------------------+-------------+-----------+-------------------+--------------------+--------+-------+--------+-------------+------------+
|n1WpP7iowLc|     17.14.11|Eminem - Walk On ...|   EminemVEVO|         10|2017-11-10 22:30:03|"Eminem"|"Walk"|"...|17158579| 787425|   43420|       125882|  2017-11-10|
|0dBIkQ4Mz1M|     17.14.11|PLUSH - Bad Unbox...|    iDubbbzTV|         23|2017-11-13 22:30:00|"plush"|"bad unbo...| 1014651| 127794|    1688|        13030|  2017-11-13|
|5qpjK5DgCt4|     17.14.11|Racist Superman |...| Rudy Mancuso|         23|2017-11-13 00:35:24|"racist superman"...| 3191434| 146035|    5339|         8181|

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,publish_date
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10 22:30:03,"""Eminem""|""Walk""|""On""|""Water""|""Aftermath/Shady/...",17158579,787425,43420,125882,2017-11-10
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13 22:30:00,"""plush""|""bad unboxing""|""unboxing""|""fan mail""|""...",1014651,127794,1688,13030,2017-11-13
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-13 00:35:24,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146035,5339,8181,2017-11-12
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 23:31:41,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",2095828,132239,1989,17518,2017-11-12
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09 16:34:14,"""edsheeran""|""ed sheeran""|""acoustic""|""live""|""co...",33523622,1634130,21082,85067,2017-11-09
...,...,...,...,...,...,...,...,...,...,...,...,...
136987,l884wKofd54,18.14.06,Enrique Iglesias - MOVE TO MIAMI (Official Vid...,EnriqueIglesiasVEVO,10,2018-05-09 12:30:01,"""Enrique Iglesias feat. Pitbull""|""MOVE TO MIAM...",25066952,268088,12783,9933,2018-05-09
136988,IP8k2xkhOdI,18.14.06,Jacob Sartorius - Up With It (Official Music V...,Jacob Sartorius,10,2018-05-11 22:39:16,"""jacob sartorius""|""jacob""|""up with it""|""jacob ...",1492219,61998,13781,24330,2018-05-11
136989,Il-an3K9pjg,18.14.06,Anne-Marie - 2002 [Official Video],Anne-Marie,10,2018-05-08 16:35:08,"""anne""|""marie""|""anne-marie""|""2002""|""two thousa...",29641412,394830,8892,19988,2018-05-08
136990,-DRsfNObKIQ,18.14.06,Eleni Foureira - Fuego - Cyprus - LIVE - First...,Eurovision Song Contest,24,2018-05-09 02:02:32,"""Eurovision Song Contest""|""2018""|""Lisbon""|""Cyp...",14317515,151870,45875,26766,2018-05-08


In [18]:
df_converted=csv_df.withColumn("publish_time", f.date_format("publish_time","HH:mm:ss"))
df_converted.show(5)

+-----------+-------------+--------------------+-------------+-----------+------------+--------------------+--------+-------+--------+-------------+------------+
|   video_id|trending_date|               title|channel_title|category_id|publish_time|                tags|   views|  likes|dislikes|comment_count|publish_date|
+-----------+-------------+--------------------+-------------+-----------+------------+--------------------+--------+-------+--------+-------------+------------+
|n1WpP7iowLc|     17.14.11|Eminem - Walk On ...|   EminemVEVO|         10|    22:30:03|"Eminem"|"Walk"|"...|17158579| 787425|   43420|       125882|  2017-11-10|
|0dBIkQ4Mz1M|     17.14.11|PLUSH - Bad Unbox...|    iDubbbzTV|         23|    22:30:00|"plush"|"bad unbo...| 1014651| 127794|    1688|        13030|  2017-11-13|
|5qpjK5DgCt4|     17.14.11|Racist Superman |...| Rudy Mancuso|         23|    00:35:24|"racist superman"...| 3191434| 146035|    5339|         8181|  2017-11-12|
|d380meD0W0M|     17.14.11|I

In [21]:
newdf=df_converted.withColumn("trending_date",from_unixtime(unix_timestamp('trending_date', 'yy.dd.MM')).cast(DateType()))
newdf.show(5)
newdf.toPandas()

+-----------+-------------+--------------------+-------------+-----------+------------+--------------------+--------+-------+--------+-------------+------------+
|   video_id|trending_date|               title|channel_title|category_id|publish_time|                tags|   views|  likes|dislikes|comment_count|publish_date|
+-----------+-------------+--------------------+-------------+-----------+------------+--------------------+--------+-------+--------+-------------+------------+
|n1WpP7iowLc|   2017-11-14|Eminem - Walk On ...|   EminemVEVO|         10|    22:30:03|"Eminem"|"Walk"|"...|17158579| 787425|   43420|       125882|  2017-11-10|
|0dBIkQ4Mz1M|   2017-11-14|PLUSH - Bad Unbox...|    iDubbbzTV|         23|    22:30:00|"plush"|"bad unbo...| 1014651| 127794|    1688|        13030|  2017-11-13|
|5qpjK5DgCt4|   2017-11-14|Racist Superman |...| Rudy Mancuso|         23|    00:35:24|"racist superman"...| 3191434| 146035|    5339|         8181|  2017-11-12|
|d380meD0W0M|   2017-11-14|I

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,publish_date
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,22:30:03,"""Eminem""|""Walk""|""On""|""Water""|""Aftermath/Shady/...",17158579,787425,43420,125882,2017-11-10
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,22:30:00,"""plush""|""bad unboxing""|""unboxing""|""fan mail""|""...",1014651,127794,1688,13030,2017-11-13
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,00:35:24,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146035,5339,8181,2017-11-12
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,23:31:41,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",2095828,132239,1989,17518,2017-11-12
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,16:34:14,"""edsheeran""|""ed sheeran""|""acoustic""|""live""|""co...",33523622,1634130,21082,85067,2017-11-09
...,...,...,...,...,...,...,...,...,...,...,...,...
136987,l884wKofd54,2018-06-14,Enrique Iglesias - MOVE TO MIAMI (Official Vid...,EnriqueIglesiasVEVO,10,12:30:01,"""Enrique Iglesias feat. Pitbull""|""MOVE TO MIAM...",25066952,268088,12783,9933,2018-05-09
136988,IP8k2xkhOdI,2018-06-14,Jacob Sartorius - Up With It (Official Music V...,Jacob Sartorius,10,22:39:16,"""jacob sartorius""|""jacob""|""up with it""|""jacob ...",1492219,61998,13781,24330,2018-05-11
136989,Il-an3K9pjg,2018-06-14,Anne-Marie - 2002 [Official Video],Anne-Marie,10,16:35:08,"""anne""|""marie""|""anne-marie""|""2002""|""two thousa...",29641412,394830,8892,19988,2018-05-08
136990,-DRsfNObKIQ,2018-06-14,Eleni Foureira - Fuego - Cyprus - LIVE - First...,Eurovision Song Contest,24,02:02:32,"""Eurovision Song Contest""|""2018""|""Lisbon""|""Cyp...",14317515,151870,45875,26766,2018-05-08


In [ ]:
import pyspark.sql.functions as f

df_converted=csv_df.select(f.date_format("publish_time","HH:mm:ss").alias("publishtime"))
df_converted.show(5)
df_converted.toPandas()

In [ ]:
schema2=StructType([StructField("publishtime", StringType(), True)])
schema2
new_csv_df = StructType(schema.fields + schema2.fields)
new_csv_df